In [21]:
from torchPIV import OfflinePIV
import torch

import numpy as np
import matplotlib.pyplot as plt

import cv2
import os

In [14]:
piv_gen = OfflinePIV(
    folder="test_imgs", # Path to experiment
    device='cpu', # Device name
    file_fmt="bmp",
    wind_size=64,
    overlap=32,
    dt=12, # Time between frames, mcs
    scale = 0.02, # mm/pix
    multipass=2,
    multipass_mode="CWS", # CWS or DWS
    multipass_scale=2.0, # Window downscale on each pass
    folder_mode="pairs" # Pairs or sequential frames 
)

results = []
for out in piv_gen():
    x, y, vx, vy = out
    results.append(out)

Load time 0.028 sec Iteration finished in 4.257 sec Batch finished in 4.955 sec
Load time 0.053 sec Iteration finished in 3.628 sec Batch finished in 4.174 sec
Load time 0.050 sec Iteration finished in 3.454 sec Batch finished in 3.995 sec
Load time 0.039 sec Iteration finished in 3.490 sec Batch finished in 4.051 sec
Load time 0.068 sec Iteration finished in 3.432 sec Batch finished in 4.003 sec
Load time 0.068 sec Iteration finished in 3.483 sec Batch finished in 4.086 sec
Load time 0.050 sec Iteration finished in 3.555 sec Batch finished in 4.108 sec
Load time 0.046 sec Iteration finished in 3.371 sec Batch finished in 3.910 sec
Load time 0.040 sec Iteration finished in 3.378 sec Batch finished in 3.929 sec
Load time 0.036 sec Iteration finished in 3.381 sec Batch finished in 3.918 sec
Load time 0.035 sec Iteration finished in 3.581 sec Batch finished in 4.200 sec


In [25]:
for frame, (x, y, vx, vy) in enumerate(results):
    plt.quiver(x, y, vx, vy)
    plt.title(frame)
    plt.show()

In [10]:
for img_file in os.listdir(r'test_imgs'):
    img = cv2.imread(rf'test_imgs/{img_file}', cv2.IMREAD_GRAYSCALE)
    xsize, ysize = img.shape
    
    newx, newy = int(xsize*0.3), int(ysize*0.3)
    img = cv2.resize(img, (newx, newy), interpolation= cv2.INTER_LINEAR)
    
    cv2.imshow('IMAGE', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()